In [ ]:
import numpy as np
import random
import time
from math import *
import matplotlib.pyplot as plt
from scipy.optimize import minimize

# Simulation Parameters

granularity = 0.25 # controls simulation time steps. 0.25 means quarter day (i.e. 6 hours). This
# means that cells will undergo births and deaths every 6 hours.

# Input Variables
# cell numbers
c = np.array([5073, 109448, 1139993, 3586752, 5621279, 5850708, 6884669, 7045420, 7289318, 5617946, 6357299, 7027523], dtype=np.float64) 
# days post-transplant
ts = np.array([9, 12, 16, 21, 27, 34, 42, 51, 61, 75, 90, 110], dtype=np.float64) 
# half-life in days
death_rate = 22.0
# convert half-life in simulation time steps (granularity).
# i.e 22.0/0.25 = 88 quarter days instead of 22 days in this case
death_rate = int(death_rate/granularity)
# convert half-life into probability
death_prob = log(2)/death_rate

In [ ]:
# Function 1: simulate cell births and deaths between ts0 and ts1. c0 is the number of cells at
# starting time ts0 and ts1 is the ending time point. birth_prob and death_prob are birth and death
# probabilities that control cell generation and cell deaths between ts0 and ts1. This function
# simulates cell births and deaths and returns the average number of cells at the ending time
# point ts1, and average number of new cells produced during ts0 to ts1

def simulate_one_transition(c0, ts0, ts1, granularity, birth_prob, death_prob):
    numpaths = 20 # For averaging purpose, used different numpaths based on the cell numbers
    # variable higher numpaths if c is lower and lower numpaths is c is higher
    path = 0
    avg_end_population = 0
    avg_new_cells = 0
    while path < numpaths:
        t = ts0
        current_population = int(c0) # Initialize current population
        new_cells = 0 # Initialize new cells for current path
        while t <= ts1: # Proceed in time-steps controlled by granularity, until we reach ts1
            count = 0
            for cell in range(current_population):
                # Generate a random variable to decide if the cell
                # undergoes birth or death or remains as it is
                s = np.random.uniform(0.0, 1.0, 1) 
                if (s[0] >= 0 and s[0] < birth_prob): # cell undergoes birth
                    count += 1
                    new_cells += 1
                elif (s[0] >= birth_prob and s[0] < birth_prob + death_prob): # cells dies
                    count -= 1
                    
            current_population = current_population + count #update the cell count
            t = t + granularity # Increment time
        avg_end_population += current_population
        avg_new_cells += new_cells
        path = path + 1
            
    return (float(avg_end_population/numpaths), float(avg_new_cells/numpaths))

In [ ]:
# Function 2: Calculates and returns the birth probability between two post-transplant days, ts0
# and ts1, such that the total number of cells at ts1 is close to c1, given that there are c0 cells at
# ts0. Function 2 considers several birth probabilities and invokes function 1 for each birth_prob
# and finally figures out which birth_prob is best

def calculate_birth_prob_single_transition(c0, c1, ts0, ts1, granularity, death_prob):
    # Try different birth probabilities, starting from 0 in
    # steps of 0.0025, with maximum of 1
    birth_probs = np.arange(0.0, 1.0, 0.0025) 
    error = []
    # For each birth probability considered (birth_probs[i]), use the simulate_one_transition() 
    # to find out the cell population at ts1
    for i in range(len(birth_probs)):
        c1_fitted, new_cells = simulate_one_transition(c0, ts0, ts1, granularity, birth_probs[i], death_prob)
        error.append((c1_fitted-c1) * (c1_fitted-c1))
        if (c1_fitted >= c1):
            break
            
    error = np.array(error, dtype=np.float64)        
    idx = np.argmin(error)
    
    # birth_probs[idx] is the best birth probability for the time period ts0 to ts1, which produces
    # necessary cell births and deaths, and results in approximately c1 cells at ts1. The function
    # returns the birth probability along with the total cells and new cells at ts1, obtained by simulation
    
    c1_fitted, new_cells = simulate_one_transition(c0, ts0, ts1, granularity, birth_probs[idx], death_prob)
    
    return ((birth_probs[idx], c1_fitted, new_cells))

In [ ]:
# Function 3: This function calculates the birth probabilities for all the post-transplant days.

def calculate_birth_prob_MDPmodel(ts, c, death_prob, granularity):
    bdp = []
    # Consider two consecutive post-transplant days ts[i], ts[i+1] and calculate birth probability
    # these two time points using calculate_birth_prob_single_transition(). 
    # And then repeat the process for all post-transplant days
    for i in range(len(ts)-1):
        c0 = c[i]
        c1 = c[i+1]
        ts0 = ts[i]
        ts1 = ts[i+1]
        ret = calculate_birth_prob_single_transition(c0, c1, ts0, ts1, granularity, death_prob)
        bdp.append(ret)        
    bdp = np.array(bdp, dtype=np.float64)
    return bdp

In [ ]:
# Invoke Function 3 using input parameters defined earlier. Extract the birth_prob values
# from returned value and convert it back to birth rate using formula birth_rate = log(2)/birth_prob

bdp = calculate_birth_prob_MDPmodel(ts, c, death_prob, granularity)
# Extract birth probability bdp[0,1] and convert print birth rate and print
print(granularity * log(2)/bdp[:,0])
# print total cells 
print(bdp[:,1])
#print new cells
print(bdp[:,2])